In [2]:
## SETUP ##
import time
from random import randint
import pandas as pd
import numpy as np
import timeit
from datetime import datetime, timedelta
from pytrends.request import TrendReq
from itertools import chain
import seaborn as sns
import string
import pickle
import matplotlib.pyplot as plt

%matplotlib inline

## Create Dictionary

In [3]:
data = pd.read_csv('/Users/mueric35/Desktop/inquireraugmented.csv', low_memory = False)

In [4]:
data = data.iloc[1:]
data = data[['Entry','Econ@','ECON','Positiv','Negativ','Pstv','Ngtv']]

In [5]:
data = data[ (data['Econ@'] == 'Econ@') | (data['ECON'] == 'ECON')]
index = (~data['Positiv'].isnull()) | ~(data['Negativ'].isnull()) | ~(data['Pstv'].isnull()) | ~(data['Ngtv'].isnull())
data = data[index]

In [6]:
temp = data[['Positiv','Negativ','Pstv','Ngtv']]
data[['Positiv','Negativ','Pstv','Ngtv']] = temp.notnull().astype(int)
data['Char'] = 0
data.loc[(data['Positiv'] == 1) | (data['Pstv'] == 1),'Char'] = 1
data.loc[~((data['Positiv'] == 1) | (data['Pstv'] == 1)),'Char'] = 0
data.loc[:,'Entry'] = data['Entry'].str.lower().str.split('#').str[0]

In [7]:
data.head()

,Entry,Econ@,ECON,Positiv,Negativ,Pstv,Ngtv,Char
51,abundance,NaN,ECON,1,0,1,0,1
97,accrue,Econ@,ECON,1,0,0,0,1
217,advantage,NaN,ECON,1,0,1,0,1
255,affluence,Econ@,NaN,1,0,0,0,1
256,affluent,Econ@,NaN,1,0,0,0,1


#### Primitive word list

In [8]:
final_dict = data[['Entry','Char']]
final_dict.reset_index(drop=True,inplace=True)
final_dict = final_dict.drop_duplicates()

In [9]:
final_dict.shape

(150, 2)

In [10]:
final_dict.head()

,Entry,Char
0,abundance,1
1,accrue,1
2,advantage,1
3,affluence,1
4,affluent,1


## Google Trend 

In [20]:
from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US', tz=360, geo = 'us')

### import primitive word 

In [21]:
# word list
pri_word = final_dict['Entry'].tolist()


In [24]:
def primitive_word(pri_word , top = 5):
    '''
    pri_word : word list
    top : top related query
    '''
    word_lst = []
    i = 0 
    for word in pri_word:
        print('Downloading: ', i)
        pytrends.build_payload([word], cat=16, timeframe='2010-01-01 2017-01-01', geo='US')

        series_queries = pytrends.related_queries()
        if     series_queries[word]['top'] is None:
            pass
        else:
            series_queries = series_queries[word]['top']['query']
            word_lst.append(series_queries[:top].tolist())

        time.sleep(randint(5, 10))
        i+=1
    return(word_lst)

In [28]:
#word_lst_5 = primitive_word(pri_word,5)
#word_lst_5
Word_lst_5 = list(chain.from_iterable(word_lst_5))

In [445]:
Word_lst = word_lst
Word_lst = list(chain.from_iterable(Word_lst))

In [654]:
# remove duplicate
Word_lst = list(set(Word_lst))
len(Word_lst)

960

### Save File 

In [770]:
Word_list = open('/Users/mueric35/Desktop/project/word_list.txt', 'w')
for word in Word_lst:
    Word_list.write("%s\n" % word)
